# 🎙️ Voice Lab GPT - Google Colab Demo

Professional voice and speech analysis system running in Google Colab.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-org/voice-lab-gpt/blob/main/notebooks/Voice_Lab_GPT_Colab.ipynb)

## Setup and Installation

In [ ]:
# Install system dependencies
!apt-get update -qq
!apt-get install -y libsndfile1 ffmpeg

# Install Python packages
!pip install numpy scipy librosa praat-parselmouth matplotlib seaborn pandas
!pip install jinja2 pyloudnorm pydub scikit-learn plotly

print("✅ Dependencies installed successfully!")

In [ ]:
# Clone Voice Lab GPT repository
!git clone https://github.com/your-org/voice-lab-gpt.git
%cd voice-lab-gpt
!pip install -e .

print("✅ Voice Lab GPT installed!")

## Quick Demo

In [ ]:
import numpy as np
from voice_lab import VoiceLabGPT
import matplotlib.pyplot as plt
from IPython.display import Audio, display, HTML
import base64
from io import BytesIO

# Initialize Voice Lab GPT
voice_lab = VoiceLabGPT(enable_visualizations=True)

print("🎙️ Voice Lab GPT initialized successfully!")

In [ ]:
# Generate synthetic voice sample for demo
def generate_demo_voice(condition='normal'):
    sr = 16000
    duration = 3.0
    t = np.linspace(0, duration, int(sr * duration))
    
    conditions = {
        'normal': {'f0': 150, 'jitter': 0.005, 'shimmer': 0.02, 'noise': 0.01},
        'breathy': {'f0': 140, 'jitter': 0.008, 'shimmer': 0.04, 'noise': 0.15},
        'rough': {'f0': 160, 'jitter': 0.030, 'shimmer': 0.12, 'noise': 0.02}
    }
    
    params = conditions[condition]
    f0 = params['f0']
    
    # Generate harmonic series with perturbations
    audio = np.zeros_like(t)
    for h in range(1, 6):
        jitter_noise = params['jitter'] * np.random.randn(len(t))
        shimmer_noise = params['shimmer'] * np.random.randn(len(t))
        
        frequency = f0 * h * (1 + jitter_noise)
        amplitude = (1.0 / h) * (1 + shimmer_noise)
        
        audio += amplitude * np.sin(2 * np.pi * frequency * t)
    
    # Add noise
    audio += params['noise'] * np.random.randn(len(t))
    
    # Normalize
    audio = audio / np.max(np.abs(audio)) * 0.8
    
    return audio, sr

# Generate and play demo audio
demo_audio, sr = generate_demo_voice('normal')

print("🎵 Generated demo voice sample:")
display(Audio(demo_audio, rate=sr))

In [ ]:
# Analyze the demo voice
print("🔬 Analyzing voice sample...")

results = voice_lab.analyze_audio_array(
    demo_audio,
    voice_type='female',
    patient_info='Demo - Normal Voice',
    task_description='Sustained /a/ vowel (synthetic)',
    generate_reports=True,
    generate_visualizations=True
)

if 'error' not in results:
    print("✅ Analysis completed successfully!")
    
    # Display key results
    grbas = results['grbas_results']
    acoustic = results['acoustic_features']
    clinical = results['clinical_results']
    
    print(f"\n📊 GRBAS: G{grbas['G']}R{grbas['R']}B{grbas['B']}A{grbas['A']}S{grbas['S']}")
    print(f"🏥 Primary Impression: {clinical['primary_impression']['condition'].replace('_', ' ').title()}")
    print(f"🎯 Confidence: {clinical['primary_impression']['confidence']:.0%}")
    
else:
    print(f"❌ Analysis failed: {results['error']}")

In [ ]:
# Display clinical summary
if 'error' not in results:
    summary = voice_lab.get_clinical_summary(results)
    print("📋 Clinical Summary:")
    print("=" * 50)
    print(summary)

In [ ]:
# Display visualizations if available
if results.get('visualizations'):
    print("📈 Generated Visualizations:")
    
    for plot_name, plot_data in results['visualizations'].items():
        if plot_name != 'error' and isinstance(plot_data, str):
            # Decode base64 image and display
            try:
                img_bytes = base64.b64decode(plot_data)
                display(HTML(f'<h3>{plot_name.replace("_", " ").title()}</h3>'))
                display(HTML(f'<img src="data:image/png;base64,{plot_data}" style="max-width: 100%; height: auto;"/>'))
            except:
                print(f"Could not display {plot_name}")

## Upload Your Own Audio File

In [ ]:
from google.colab import files
import librosa

# Upload audio file
print("📁 Upload your audio file (WAV, MP3, etc.):")
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    print(f"✅ Uploaded: {filename}")
    
    # Load and play the uploaded audio
    audio, sr = librosa.load(filename, sr=None)
    display(Audio(audio, rate=sr))
    
    # Analyze the uploaded audio
    print("🔬 Analyzing uploaded audio...")
    
    user_results = voice_lab.analyze_audio_array(
        audio,
        voice_type=None,  # Auto-detect
        patient_info=f'User Upload - {filename}',
        task_description='User provided audio',
        generate_reports=True,
        generate_visualizations=True
    )
    
    if 'error' not in user_results:
        print("✅ Analysis completed!")
        
        # Display results
        summary = voice_lab.get_clinical_summary(user_results)
        print("\n📋 Your Audio Analysis:")
        print("=" * 50)
        print(summary)
        
    else:
        print(f"❌ Analysis failed: {user_results['error']}")

## Compare Different Voice Conditions

In [ ]:
# Generate and analyze different voice conditions
conditions = ['normal', 'breathy', 'rough']
condition_results = {}

print("🔬 Analyzing different voice conditions...")

for condition in conditions:
    print(f"\nAnalyzing {condition} voice...")
    
    # Generate audio for this condition
    audio, sr = generate_demo_voice(condition)
    
    # Analyze
    results = voice_lab.analyze_audio_array(
        audio,
        voice_type='female',
        patient_info=f'Demo - {condition.title()} Voice',
        generate_reports=False,
        generate_visualizations=False
    )
    
    condition_results[condition] = results
    
    if 'error' not in results:
        grbas = results['grbas_results']
        clinical = results['clinical_results']
        
        grbas_str = f"G{grbas['G']}R{grbas['R']}B{grbas['B']}A{grbas['A']}S{grbas['S']}"
        impression = clinical['primary_impression']['condition'].replace('_', ' ').title()
        confidence = clinical['primary_impression']['confidence']
        
        print(f"  GRBAS: {grbas_str}")
        print(f"  Impression: {impression} ({confidence:.0%})")
        
        # Play audio sample
        display(HTML(f'<h4>{condition.title()} Voice Sample:</h4>'))
        display(Audio(audio, rate=sr))
    
    else:
        print(f"  ❌ Analysis failed: {results['error']}")

print("\n✅ Condition comparison completed!")

## Export Results

In [ ]:
# Save reports and download
if 'results' in locals() and 'error' not in results:
    # Create reports directory
    !mkdir -p reports
    
    # Save reports
    saved_files = voice_lab.save_reports(
        output_dir='./reports',
        results=results,
        formats=['json', 'html', 'clinical_summary']
    )
    
    print("📁 Reports saved:")
    for format_type, filepath in saved_files.items():
        print(f"  ✓ {format_type}: {filepath}")
    
    # Download files
    print("\n⬇️ Downloading reports...")
    for filepath in saved_files.values():
        files.download(filepath)
    
    print("✅ Reports downloaded to your computer!")

## Next Steps

🎉 **Congratulations!** You've successfully run Voice Lab GPT in Google Colab.

### What you can do next:
- Upload your own voice recordings for analysis
- Experiment with different voice conditions
- Download and examine the generated reports
- Integrate Voice Lab GPT into your research or clinical workflow

### For production use:
- Install Voice Lab GPT on your local machine or server
- Set up batch processing for multiple files
- Integrate with your existing clinical systems

### Resources:
- [GitHub Repository](https://github.com/your-org/voice-lab-gpt)
- [Documentation](https://voice-lab-gpt.readthedocs.io)
- [Research Papers](https://voice-lab-gpt.readthedocs.io/references)

---

**🤖 Generated with Voice Lab GPT**

*Professional voice analysis for clinical excellence*